This notebook generates position data with noise for orbit determination purposes, using a Keplerian propagator. 

Parameters for data generation

In [1]:
import numpy as np
import datetime
orbit_epoch = datetime.datetime(2020, 1, 1)
sma = 10000e3  # Semi-major axis in meters
ecc = 0.2  # Eccentricity
inc = float(np.deg2rad(60.0))  # Inclination in radians
raan = 0.0  # Right-ascension of ascending node in radians
aop = 0.0  # Argument of perigee in radians
ta = 0.0  # Initial true anomaly in radians

sigma_position = 100e3  # Noise (in terms of standard deviation of gaussian distribution) of output position data in meters
sigma_velocity = 100.0  # Noise of output velocity data in meters per second

n_orbits = 1.0  # Number of orbits to generate
T = 500.0  # Sample time of output data in seconds

orbit_period = 2*np.pi*np.sqrt(sma**3 / 3.98600e14)
duration = float(n_orbits*orbit_period)
display(duration)

9952.019565792982

Firing up a JVM for Orekit

In [2]:
import orekit
orekit.initVM()

Downloading and importing the Orekit data ZIP

In [3]:
from orekit.pyhelpers import download_orekit_data_curdir, setup_orekit_curdir
download_orekit_data_curdir()
setup_orekit_curdir()

Setting up models (frames, timescales)

In [4]:
from org.orekit.frames import FramesFactory
gcrf = FramesFactory.getGCRF()

Setting up initial orbit and propagator

In [5]:
from org.orekit.orbits import KeplerianOrbit, PositionAngle
from orekit.pyhelpers import datetime_to_absolutedate
from org.orekit.utils import Constants as orekit_constants
orbit = KeplerianOrbit(sma, ecc, inc, aop, raan, ta,
                       PositionAngle.TRUE, 
                       gcrf,
                       datetime_to_absolutedate(orbit_epoch),
                       orekit_constants.EIGEN5C_EARTH_MU)

from org.orekit.propagation.analytical import KeplerianPropagator
propagator = KeplerianPropagator(orbit)

Orekit can generate measurements for all types of measurements that are used for OD. However for position/velocity, it is easy to retrieve the measurements by hand.

Propagating, adding noise and saving data to a pandas dataframe.

In [6]:
import pandas as pd
position_df = pd.DataFrame(columns=['x', 'y', 'z', 'vx', 'vy', 'vz'])
position_without_noise_df = pd.DataFrame(columns=['x', 'y', 'z', 'vx', 'vy', 'vz'])

from orekit.pyhelpers import absolutedate_to_datetime
date_start = datetime_to_absolutedate(orbit_epoch)
date_end = date_start.shiftedBy(duration)
date_current = date_start

while date_current.compareTo(date_end) < 0:
    pv_gcrf = propagator.getPVCoordinates(date_current, gcrf)
    
    # Adding noise to position and velocity
    pos_without_noise = np.array(pv_gcrf.getPosition().toArray())
    pos_with_noise = pos_without_noise + np.random.normal(0, sigma_position, len(pos_without_noise))
    vel_without_noise = np.array(pv_gcrf.getVelocity().toArray())
    vel_with_noise = vel_without_noise + np.random.normal(0, sigma_velocity, len(vel_without_noise))
    
    position_df.loc[absolutedate_to_datetime(date_current)] = np.concatenate(
        (pos_with_noise, vel_with_noise)
    )
    position_without_noise_df.loc[absolutedate_to_datetime(date_current)] = np.concatenate(
        (pos_without_noise, vel_without_noise)
    )
    date_current = date_current.shiftedBy(T)
    
display(position_df)

,x,y,z,vx,vy,vz
2020-01-01 00:00:00,8.056982e+06,1.468446e+04,3.100794e+04,-41.522207,3885.734219,6677.381032
2020-01-01 00:08:20,7.154730e+06,1.852806e+06,3.170712e+06,-2908.078162,3434.859916,6098.294914
2020-01-01 00:16:40,5.006926e+06,3.606465e+06,5.901557e+06,-5315.241457,2799.273333,4452.587151
2020-01-01 00:25:00,2.352366e+06,4.434280e+06,7.593867e+06,-6313.109418,1530.104869,2432.364617
2020-01-01 00:33:20,-1.007620e+06,5.033059e+06,8.323170e+06,-6278.175584,296.658983,754.782685
2020-01-01 00:41:40,-4.067582e+06,4.962326e+06,8.267137e+06,-5918.025799,-525.722689,-1004.132238
2020-01-01 00:50:00,-6.908941e+06,4.381691e+06,7.569041e+06,-5101.186402,-1353.391861,-2258.428265
2020-01-01 00:58:20,-8.900902e+06,3.333955e+06,5.922577e+06,-3865.802334,-1957.304014,-3284.611687
2020-01-01 01:06:40,-1.072952e+07,2.449153e+06,4.063725e+06,-2690.374787,-2389.261654,-3898.051694
2020-01-01 01:15:00,-1.170893e+07,1.477098e+06,2.010125e+06,-1299.566595,-2455.605459,-4262.773492


Finally, saving position data to CSV file

In [7]:
position_df.to_csv('pos_vel_data_gcrf_with_noise.csv')
position_without_noise_df.to_csv('pos_vel_data_gcrf_without_noise.csv')